In [1]:
#! git clone https://github.com/HSE-LAMBDA/RheologyReconstruction.git
#! pip install 'dvc[azure]'
#! pip install pyevtk

In [2]:
#! cd RheologyReconstruction/datasets && dvc pull velocity_models_china.tar
#! tar xvf /content/RheologyReconstruction/datasets/velocity_models_china.tar

In [3]:
import os
import sys

sys.path.append('/content/RheologyReconstruction')

In [4]:
! mkdir dataset/
! mkdir dataset/seismograms/
! mkdir dataset/masks/

mkdir: cannot create directory ‘dataset/’: File exists
mkdir: cannot create directory ‘dataset/seismograms/’: File exists
mkdir: cannot create directory ‘dataset/masks/’: File exists


In [5]:
import numpy as np
from tqdm.notebook import tqdm
from pipeline.acoustics_solver import acoustics_solver

vmodels_dir = '/content/velocity_models_china/'
dataset_dir = '/content/dataset/'

buffer = _

for i, model in tqdm(enumerate(sorted(os.listdir(vmodels_dir))), desc='running'):

    cp = np.load(vmodels_dir + model)

    # marmousi-style density from cp

    rho = np.zeros_like(cp)
    rho[cp < 4500.  ] = cp[cp < 4500. ] * 0.2806 + 265.
    rho[cp >= 4500. ] = 2140 

    x_size = 3000.
    y_size = 2000.
    
    solver = acoustics_solver(
        x_size, y_size, 
        cp.T[:, ::-1], rho.T[:, ::-1], 
        2.0, 0.01, 
        100., # source width
        source_center_in_percents=50., 
        dump_vtk=False, 
        verbose=False
    )
    
    buffer = solver.forward() 
    np.save(os.path.join(dataset_dir, 'masks', f'{i}'), cp)
    np.save(os.path.join(dataset_dir, 'seismograms', f'{i}'), buffer[::-1])